# Recommendation System 

In [1]:
import pandas as pd
import numpy as np

datafile = "./ml-100k/u.data"
data = pd.read_csv(datafile, sep = '\t', header = None, names = ['userId', 'itemId', 'rating', 'timestamp'])
# header = None mean the file has no col name so col number will be asssigned.
data.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
movieInfoFile = './ml-100k/u.item'
movieInfo = pd.read_csv(movieInfoFile, sep = '|', header = None, index_col = False, encoding='latin-1',
                        names = ['itemId', 'title'], usecols = [0, 1])

# the datset was giving utf-8 encoding error therefore i used latin-1 encoding, on which the dataset worked perfectly ok
# usecols mean dont use all the columns present in the dataset. just use 0 and 1 column. 
# if we dont set index_col = False then pandas by default would have taken itemId col as an row index no col. 

movieInfo.head()

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [3]:
# now merging both the tables with a common col name in both the dataframes

data = pd.merge(data, movieInfo, left_on='itemId', right_on= 'itemId')

data.head()

,userId,itemId,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [4]:
data.loc[0: 10, ['userId']]
# loc will give u 11 values from the userId col. 

,userId
0,196
1,63
2,226
3,154
4,306
5,296
6,34
7,271
8,201
9,209


In [7]:
# for finding the max userId and itemId

numUsers = max(data.userId)
numMovies = max(data.itemId)

# we can also see how many movies were rated by each user and the number of users that rated each movie

moviesPerUser = data.userId.value_counts()
# this will tell how many times each user appears in the dataframe. i.e how many movies each user rated. 
usersPerMovie = data.title.value_counts()
# this will tell how many times each title appears in the dataframe. i.e. how many users rated each movie 

print(moviesPerUser)
print("-------------")
print(usersPerMovie)

In [8]:
# lets write a function to find top N favourite movies of a user.

def favoriteMovies(activeUser, N):
    user = data[data.userId == activeUser]
    top_movies = pd.DataFrame.sort_values(user, ['rating'], ascending = [0])[:N]
    # 0 mean sort in descending order.
    # fetch all the records/rows till N. 
    return list(top_movies.title) # we only need title col of that user. and this will return a list not df.

print(favoriteMovies(5, 3))  # 5 is the userId and 3 is the total top 3 rated movies by that user.

['Empire Strikes Back, The (1980)', 'Blade Runner (1982)', 'Wrong Trousers, The (1993)']


### let's get down to finding some recommendations now.

In [9]:
# we will represent each user as a vector. each element in the vector will be the rating of the specific movie 
# by that user. since we have 1600 odd movies so each vector will contain 1600 elements. 

userItemRatingMatrix = pd.pivot_table(data, values = 'rating', index= ['userId'],
                                     columns=['itemId'])
# each row will be a user and cols will be rating for each movie
# index hoga userId and uski values hongi ratings. cols name will be itemId.
userItemRatingMatrix.head()

itemId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Latent Factor Based Model for Recommendation System
### It uses Matrix Factorization

In [10]:
# The idea here is to identify some hidden factors which influence the user rating of a movie.
# The way to do this is by decomposing user item matrix into 2 other matrices. One will be a user factor matrix
# and other will be an item-factor matrix. 
# each row in the user factor matrix maps the user onto the set of hidden factors 
# each col in the item factor matrix maps the item on to the hidden factors. 
# in the matrix factorization u find all the ratings for all the users in one go. 

In [11]:
def matrixFactorization(R, K, steps = 10, gamma= 0.001, lamda = 0.02):
    """
    inputs
      - R: user item rating matrix
      - K: the no of factors k
      - steps, gamma and lambda are used to optimize our error fucntion and find the right factor vectors
    """
    # we will be using sgd to find the factors vectors
    # SGD will use steps, gamma and lambda
    
    N = len(R.index) # Number of users
    M = len(R.columns) # Number of items
    P = pd.DataFrame(np.random.rand(N, K), index = R.index)
    # now the matrix R will be decomposed into 2 matrices. i.e P and Q
    # P is the user factor matrix and Q is the product factor matrix
    # dim of P: N x K where N is the no of users and K is the no of factors
    # we will initialize this matrix with some random numbers. then we will iteratively adjust the values of P
    # so that it moves to a place where the dot product of P and Q will be very close to user Item matrix R.
    
    Q = pd.DataFrame(np.random.rand(M, K), index = R.columns)
    # Q is the product factor matrix. Its dim will be M x K. 
    # Now we have P and Q with some initial values 
    # we need to find the final values of P and Q with SGD
    for step in range(steps):
        # find the slope and find errors at each point and move downward in the direction of slope.
        # steps will specify how many time will SGD will loop through each rating which is in the training set.
        for i in R.index:
            for j in R.columns:
                if R.loc[i, j] > 0:
                    # check for each user and his each rating is > 0 and if it exist then it will compute the 
                    # error for that rating. 
                    eij = R.loc[i, j] - np.dot(P.loc[i], Q.loc[j])
                    # this is the error difference blw the actual rating and predicted rating. 
                    # the predicted rating will be the dot product of corresponding user factor vector and the 
                    # corresponding product/item factor vector.
                    # now we should adjust the value of p and q so that we r moving towards the minimum
                    # first we need to find the slope of current point
                    P.loc[i] = P.loc[i] + gamma * (eij * Q.loc[j] - lamda * P.loc[i])
                    # (eij * Q.loc[j] - lamda * P.loc[i]) the values in the bracket is the partial derivative
                    # of the error function at this particular point. 
                    Q.loc[j] = Q.loc[j] + gamma * (eij * P.loc[i] - lamda * Q.loc[j])
        # at the end of this we have looped through all of the ratings. 
        # Let's check the current value  of the error. 
        # if the error value is less than 0.001 else we will continue
        error = 0
        for i in R.index:
            for j in R.columns:
                if R.loc[i, j] > 0:
                    # computing error
                    e = error + pow(R.loc[i, j]- np.dot(P.loc[i], Q.loc[j]), 2)
                    error = e + lamda*(pow(np.linalg.norm(P.loc[i]), 2)+ pow(np.linalg.norm(Q.loc[j]), 2))
                    # we r taking sum of the squares of the error. We will add lamda regularizer as well
                    # we use norm from numpy module to find magnituide of particular vector. 
        if error < 0.001:
            break
        print("Step No : ", step)
    return P, Q



In [12]:
# Let's call this fucntion now 
(P, Q) = matrixFactorization(userItemRatingMatrix, K = 2, gamma = 0.001, lamda = 0.2, steps = 5)
# this will return 2 matrices 
# 1 will be product factor matrices and another will be user factor matrix
# ideally we should run it over the entire matrix for few thousand steps. But it can take few hours. 
# for instance we can perform the computation over small part of the rating matrix to see how it works. 
# we can do this by saying userItemRatingMatrix[: 100, : 100] i.e.  1st hundred users and 1st hundred items


Step No :  0
Step No :  1
Step No :  2
Step No :  3
Step No :  4


In [16]:
# Let's quickly use these ratings to find recommendations for a user. 
# if u take any user i and want to find their rating for product j, u would take the ith row of matrix P and jth row of matrix Q and take a dot product of those
# 2 vectors. 
# if u want to find the predictive item rating for all the items for an  active user, take the row corresponding to the active user from the matrix P and do a dot 
# product with entire matrix Q. 
activeUser = 1
N = 5
predictItemRating = pd.DataFrame(np.dot(P.loc[activeUser], Q.T), index = Q.index, columns = ['Rating'])

# now we will remove the movies that the user has allready rated. 
# Or lets just drop the rating for the movies which the user has allready watched. 
single_row = userItemRatingMatrix.loc[activeUser]
# single_row
moviesAllreadyWatched = list(single_row.loc[single_row > 0].index) 
predictItemRating = predictItemRating.drop(moviesAllreadyWatched)
topRecommendations = pd.DataFrame.sort_values(predictItemRating,
                                                  ["Rating"], ascending = [0])[:N]

topRecommendationTitles = movieInfo.loc[movieInfo.itemId.isin(topRecommendations.index)]

print(favoriteMovies(activeUser, N))
print()
print(list(topRecommendationTitles.title))


['Kolya (1996)', 'Contact (1997)', 'Dolores Claiborne (1994)', '12 Angry Men (1957)', 'Star Trek: The Wrath of Khan (1982)']

['L.A. Confidential (1997)', 'Titanic (1997)', "Schindler's List (1993)", "One Flew Over the Cuckoo's Nest (1975)", 'Casablanca (1942)']


# Storing P and Q values

In [ ]:
P.to_csv("Q_values.csv", sep='\t')
Q.to_csv("Q_values.csv", sep='\t')